In [ ]:
# old funciton to long epochs of MNE files

def load_data(path, start, stop):
    """
    Uses MNE-toolbox to load the data. Apply noth filter at 50 Hz
    Input: path to file, start and stop in seconds as float
    output: the MNE.io.Raw object that contains the given time intervall
    """
    # check if path exist and then load the object with mne
    if os.path.exists(data_dir):

        raw = mne.io.read_raw_edf(path, exclude=['-', 'T8-P8', '.'], verbose=False, preload=True)

        # apply notch filter around 50 Hz.
        raw.notch_filter(freqs=[49.1, 50.9])

        # Stim channel is automatically created by MNE.
        #raw.drop_channels(['STI 014'])

        # Take 30 seconds before onset
        #start = start - 
        #stop = start + 150
        # check if the recording is long enough to create the feature vector
        if raw.n_times // 256 > start + 150:
            try:
                cropped = raw.crop(start, stop, verbose=False)
            except:
                # Sometime something goes wrong so I manually extract the seizure of the timewindow and return a
                # newly created object with the info
                manual_raw = raw.get_data()
                freq = raw.info['sfreq']
                manual_start = int(start * freq)
                manual_stop = int((stop * freq + 1))
                manual_data = np.asarray([manual_raw[i][manual_start:manual_stop] for i in range(len(manual_raw))])
                cropped = mne.io.RawArray(manual_data, raw.info, verbose=False)

        return cropped

raw_data = load_data('data/chb-mit-scalp-eeg-database-1.0.0/chb06/chb06_06.edf',0,2000)

In [ ]:
raw_data.plot(duration=500, n_channels=10,scalings='auto',lowpass=True)
plt.show()

In [ ]:
raw_data.plot_psd(fmax=50,average=True)
plt.show()

In [ ]:
# selecting from raw data

data, times = raw_data.get_data(return_times=True) # data: channel x time

# index method
# data = raw[:][0]
# times = raw[:][1]

# pick channels
# two_meg_chans_data = raw.get_data(picks=['MEG_0712', 'MEG_1022'],
#                                  start=1000, stop=2000)

data.shape

In [ ]:
# example crop

raw_data.copy().crop(tmin=10, tmax=12.5)

In [7]:
## check for intersect of picked channels and channels in the raw file, 17 is correct

# loading multiple
import utils
from pathlib import Path
import mne
from tqdm import tqdm

data_dir = Path(r'C:\data\chb-mit-scalp-eeg-database-1.0.0')

df_info = utils.get_summaries(data_dir)

channels =  {'FP1-F7', 'F7-T7', 'T7-P7', 'P7-O1', 'FP1-F3', 'F3-C3',
'C3-P3', 'P3-O1', 'FP2-F4', 'F4-C4', 'C4-P4', 'P4-O2', 'FP2-F8', 'F8-T8', 'P8-O2', 'FZ-CZ', 'CZ-PZ'} # 'T8-P8'

inter=[]
#for i,row in df_info.loc[25:25].iterrows():
for i,row in tqdm(df_info.iterrows()):
    path = row.folder / row.file_name
    raw = mne.io.read_raw_edf(path, verbose=False, exclude=['-', 'T8-P8', '.'], preload=False)
    raw_ch = set(raw.info['ch_names'])
    inter.append(len(raw_ch & channels))

df_info["intersect"] = inter


#data = raw.get_data(picks=channels)

data.shape

664it [00:06, 95.01it/s] 


(17, 595200)

In [9]:
df_info.query('intersect != 17')

,file_name,file_start_time,file_end_time,number_of_seizures_in_file,subject,seizure_times,folder,intersect
343,chb12_27.edf,16:07:06,17:07:06,6,chb12,"[[916, 951], [1097, 1124], [1728, 1753], [1921...",C:\data\chb-mit-scalp-eeg-database-1.0.0\chb12,0
344,chb12_28.edf,17:07:12,18:07:12,1,chb12,"[[181, 215]]",C:\data\chb-mit-scalp-eeg-database-1.0.0\chb12,0
345,chb12_29.edf,18:07:19,19:07:43,6,chb12,"[[107, 146], [554, 592], [1163, 1199], [1401, ...",C:\data\chb-mit-scalp-eeg-database-1.0.0\chb12,0


In [ ]:
# create array of power line noise + harmonics

np.arange(60, 241, 60)

In [ ]:
# info keys

list(raw.info.keys())

# Segments

In [ ]:
fs = raw_data.info['sfreq']
start_stop_seconds = np.array([0, 5])
start_sample, stop_sample = (start_stop_seconds * fs).astype(int)
raw_selection = raw[0, start_sample:stop_sample] # can do by index or name.

x = raw_selection[1]
y = raw_selection[0].T
plt.plot(x, y)

In [ ]:
win_len = 5
fs = raw.info['sfreq']
n_segments = data.shape[1]/(win_len*fs)
n_chan = raw.info["nchan"]

D = np.reshape(data,(n_chan,n_segments,-1)) # channel x segment x time 

# Tests

In [ ]:
# testing all rows to find errors
fs = 256
for i,row in df_info.iterrows():

    if int(row.number_of_seizures_in_file) > 0:

        for seizures in row.seizure_times:
            print(row.file_name)
            print(row.seizure_times)
            s, e = (seizures.astype('int') * fs).astype(int)

        

